In [25]:
import numpy as np
import sqlite3
from scipy.spatial import distance #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html


In [172]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs where "Huvudgrupp" = "Grönsaker"')
for row in rows:
        result.append(row)

db_contents = np.array(result)

conn.close()

## Prototypiska i sin kategori
Syftet är att ta en huvudgrupp och se vad mittpunkten i den kategorin är och sen räkna ut vilka livsmedel som ligger närmast mittpunkten.

In [185]:
db_contents.shape

(94, 63)

In [196]:
columns = [2,4,5,6]
# [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]


In [197]:
dataset = db_contents[:,columns].astype(float)

In [198]:
dataset

array([[  1.97200000e+02,   1.70000000e+00,   1.96000000e+01,
          1.94000000e+00],
       [  1.33000000e+01,   1.50000000e+00,   1.00000000e-01,
          7.00000000e-01],
       [  2.44000000e+01,   2.60000000e+00,   1.70000000e-01,
          1.94000000e+00],
       [  2.38000000e+01,   2.30000000e+00,   3.00000000e-01,
          2.00000000e+00],
       [  3.52000000e+01,   3.12000000e+00,   2.70000000e-01,
          3.50000000e+00],
       [  2.40000000e+01,   1.80000000e+00,   3.00000000e-01,
          2.38000000e+00],
       [  4.36000000e+01,   4.70000000e+00,   3.00000000e-01,
          3.40000000e+00],
       [  4.14000000e+01,   3.40000000e+00,   4.00000000e-01,
          3.80000000e+00],
       [  1.49000000e+01,   1.00000000e+00,   2.00000000e-01,
          1.20000000e+00],
       [  1.36000000e+01,   1.10000000e+00,   1.00000000e-01,
          1.00000000e+00],
       [  2.85000000e+01,   1.80000000e+00,   4.00000000e-01,
          2.80000000e+00],
       [  4.00000000e

In [199]:
means = np.mean(dataset, axis=0)

In [200]:
means

array([ 42.34255319,   5.1756383 ,   1.07510638,   1.88117021])

In [201]:
dist = distance.cdist(dataset, [means], 'euclidean') #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

In [202]:
dist.T[0]

array([ 156.00026935,   29.31426629,   18.14915442,   18.78058798,
          7.64921702,   18.67334498,    2.17141385,    2.8598906 ,
         27.78055948,   29.05980876,   14.29374792,    3.43189418,
         13.00402539,   29.42098675,   28.00761401,    9.96069701,
         13.30227367,    4.13692195,   84.58757226,   70.87808559,
         57.62187339,   21.31698543,   12.89885546,   24.00289184,
         20.03707218,    7.43082394,   16.31408724,   12.92467185,
         11.00594152,   28.45367223,   14.06564533,   21.91272437,
         25.24905719,   22.7372807 ,   28.05096152,   25.0165344 ,
          5.1818439 ,   17.03990485,   12.91017015,  120.27299539,
         23.66268819,   15.66363507,   15.42983487,   15.18199297,
         17.99557729,    6.44606063,   18.93599763,   17.01873173,
         13.93105958,   26.8255292 ,   20.34659153,   18.47284827,
         38.24870794,   86.40456895,  100.56737492,   27.36614355,
         25.77276098,   24.96761894,   43.24969162,   45.68824

In [203]:
order = np.argsort(dist.T[0])

In [204]:
order

array([93,  6,  7, 76, 11, 17, 66, 64, 36, 90, 45, 25,  4, 15, 77, 28, 65,
       22, 38, 27, 12, 70, 68, 16, 48, 30, 10, 81, 67, 83, 43, 42, 69, 41,
       26, 47, 37, 86, 85, 73, 44,  2, 62, 51, 71,  5,  3, 87, 46, 24, 60,
       50, 92, 21, 31, 33, 61, 40, 84, 23, 57, 35, 32, 63, 56, 82, 49, 88,
       55,  8, 14, 75, 34, 29,  9,  1, 13, 89, 72, 79, 74, 52, 58, 59, 20,
       78, 19, 18, 53, 91, 54, 39, 80,  0])

In [206]:
db_contents[order,0]

array(['Brysselkål kokt', 'Brysselkål', 'Brysselkål fryst',
       'Ättiksgurka u lag', 'Grönkål', 'Lök gul', 'Lök kokt',
       'Gurka färskinlagd', 'Trädgårdskrasse', 'Lök röd',
       'Chilipeppar färsk', 'Persilja blad', 'Broccoli', 'Kronärtskocka',
       'Kronärtskocka kokt', 'Rödkål', 'Lök stekt', 'Nässlor förvällda',
       'Vitkål', 'Purjolök', 'Grönkål fryst', 'Vitkål kokt',
       'Purjolök kokt', 'Kålrabbi', 'Lök gul fryst', 'Savojkål', 'Fänkål',
       'Fänkål kokt', 'Paprika förvälld', 'Rucolasallat', 'Gräslök',
       'Dill', 'Spenat fräst', 'Sparris grön el vit', 'Pumpa',
       'Paprika grön röd fryst', 'Vattenkrasse', 'Broccoli kokt',
       'Småtomater röda typ körsbärstomat', 'Gurka inlagd',
       'Basilika färsk', 'Blomkål', 'Blomkål kokt',
       'Bambuskott konserv u lag', 'Aubergine kokt', 'Broccoli fryst',
       'Blomkål fryst', 'Spenat färsk', 'Paprika gul', 'Paprika röd',
       'Tomater krossade konserv m lag', 'Paprika grön gul röd',
       'Sparris grön 